In [22]:
ENV["LINES"], ENV["COLUMNS"] = 100, 100
using BenchmarkTools
using Distributions
using LinearAlgebra
using Printf
using QuadGK
using Random
Random.seed!(4649373)
using Roots
using SpecialFunctions
using StaticArrays
using StatsBase
using StatsFuns
using StatsPlots
default(fmt = :png, titlefontsize = 10, size = (400, 250))
using SymPy

In [23]:
# Override the Base.show definition of SymPy.jl:
# https://github.com/JuliaPy/SymPy.jl/blob/29c5bfd1d10ac53014fa7fef468bc8deccadc2fc/src/types.jl#L87-L105

@eval SymPy function Base.show(io::IO, ::MIME"text/latex", x::SymbolicObject)
    print(io, as_markdown("\\displaystyle " * sympy.latex(x, mode="plain", fold_short_frac=false)))
end
@eval SymPy function Base.show(io::IO, ::MIME"text/latex", x::AbstractArray{Sym})
    function toeqnarray(x::Vector{Sym})
        a = join(["\\displaystyle " * sympy.latex(x[i]) for i in 1:length(x)], "\\\\")
        """\\left[ \\begin{array}{r}$a\\end{array} \\right]"""
    end
    function toeqnarray(x::AbstractArray{Sym,2})
        sz = size(x)
        a = join([join("\\displaystyle " .* map(sympy.latex, x[i,:]), "&") for i in 1:sz[1]], "\\\\")
        "\\left[ \\begin{array}{" * repeat("r",sz[2]) * "}" * a * "\\end{array}\\right]"
    end
    print(io, as_markdown(toeqnarray(x)))
end

In [24]:
safemul(x, y) = x == 0 ? x : x*y
safediv(x, y) = x == 0 ? x : x/y

x ⪅ y = x < y || x ≈ y

mypdf(dist, x) = pdf(dist, x)
mypdf(dist::DiscreteUnivariateDistribution, x) = pdf(dist, round(Int, x))

distname(dist::Distribution) = replace(string(dist), r"{.*}" => "")
myskewness(dist) = skewness(dist)
mykurtosis(dist) = kurtosis(dist)
function standardized_moment(dist::ContinuousUnivariateDistribution, m)
    μ, σ = mean(dist), std(dist)
    quadgk(x -> (x - μ)^m * pdf(dist, x), extrema(dist)...)[1] / σ^m
end
myskewness(dist::MixtureModel{Univariate, Continuous}) = standardized_moment(dist, 3)
mykurtosis(dist::MixtureModel{Univariate, Continuous}) = standardized_moment(dist, 4) -3

mykurtosis (generic function with 2 methods)

[2]

(1)

In [25]:
# n回中k回成功というデータが得られたとする
n, k = 20, 5


(20, 5)

In [26]:
# 有意水準を1%に設定 (α は \alpha TAB で入力できる)
α = 0.01

0.01

In [27]:
# 二項分布Binomial(n,p)でk以上になる確率が2.5%になるp
p_L = quantile(Beta(k, n-k+1), α/2)

0.058333936058951805

In [28]:
# 検算: 分布 Binomial(n, p_L) で k 以上になる確率
ccdf(Binomial(n, p_L), k-1)

0.004999999999999996

In [29]:
# 二項分布Binomial(n,p)でk以下になる確率が2.5%になるp
p_U = quantile(Beta(k+1, n-k), 1 - α/2)

0.5597609077584391

In [30]:
# 検算: 分布 Binomial(n, p_U) で k 以下になる確率
cdf(Binomial(n, p_U), k) # k

0.005000000000000005

In [31]:
# n回中k回成功というデータから決まるClopper-Pearsonの信頼区間
@show [p_L, p_U];

[p_L, p_U] = [0.058333936058951805, 0.5597609077584391]


In [32]:
@show round.([p_L, p_U]; digits=10);

round.([p_L, p_U]; digits = 10) = [0.0583339361, 0.5597609078]


(2)

In [33]:
n,p_0,k=20,0.5,5

(20, 0.5, 5)

In [34]:
min(1,2cdf(Binomial(n,p_0),k))

0.041389465332031285

In [35]:
@show round.(min(1,2cdf(Binomial(n,p_0),k));digits=10);

round.(min(1, 2 * cdf(Binomial(n, p_0), k)); digits = 10) = 0.0413894653


[3]

In [36]:
@vars n

(n,)

In [37]:
@show A(n)=(1/n)*((factorial(2n)/factorial(n))^(1/n))

A(n) = begin
        #= In[37]:1 =#
        (1 / n) * (factorial(2n) / factorial(n)) ^ (1 / n)
    end = A


A (generic function with 1 method)

In [38]:
@show A_limit=limit(A(n),n,oo)

A_limit = limit(A(n), n, oo) = 4*exp(-1)


   -1
4⋅ℯ  

In [39]:
float(A_limit)

1.471517764685769286382095080645843469783244524127071338031347206789845982979593

In [40]:
@show round.(float(A_limit);digits=10);

round.(float(A_limit); digits = 10) = 1.471517764700000000000000000000000000000000000000000000000000000000000000000001


1.47152が正解っぽい?

In [41]:
x=symbols("x")

x

In [42]:
limit(exp(x)/factorial(x),x,oo)

0